#### Get the maximum value from Burned Area Monthly tiles

Data source: https://lpdaac.usgs.gov/products/mcd64a1v006/


resources:
https://blog.dask.org/2019/06/20/load-image-data

https://examples.dask.org/applications/satellite-imagery-geotiff.html

In [26]:
from osgeo import gdal, osr
import os
import numpy as np
import dask.array as da
import time
import dask
import imageio
import glob
import sys

In [2]:
def ListofExtensionAndName(directory,extension):
     
        if len(directory) != None:
            import os
            FilesList = []
            FileName=[]
            for root, subdirectory, files in os.walk(directory):
                for file in files:
                    if file.endswith(extension):
                        FilesList.append(os.path.join(root,file))
                        base=os.path.basename(file)
                        FileName.append(os.path.splitext(base)[0])

            return sorted(FilesList),sorted(FileName)
        else:
            print("no"+ extension +"file for this directory")

In [3]:
folder_list=glob.glob('../data/Modis_Fire_Date/*/*/')
folder_list=sorted(folder_list)


In [5]:
folder_list_copy=folder_list.copy()
year_list=[]
for index,folder in enumerate(range(len(folder_list)//12)):
    try:
        year_list.append(folder_list_copy[0:12])
        del folder_list_copy[0:12]
    except:
        year_list.append(folder_list_copy)

        

In [12]:
yearly_imgs_list = [[sorted(glob.glob(folder+'/tif/*.tif')) for folder in year] for year in year_list]


In [51]:
%%time
for index,year in enumerate(yearly_imgs_list):
    for index2 in range(268):
        image_status=False
        counter=0
        #define target year
        year_info=year[0][0].split('/')[3]
        #under defined year, we check every mounth with the same MODIS tile nane
        area_code=yearly_imgs_list[0][0][index2].split('.')[-4]
        
        target_imgs=[]
        target_imgs=glob.glob('../data/Modis_Fire_Date/{}/*/tif/*{}*'.format(year_info,area_code))
        #print(area_code)
        if len(target_imgs)==0:
            continue
        
        while image_status is False:
            try:
                array_list=[]
                for index,img in enumerate(target_imgs):
                    ds = gdal.Open(img)
                    #print(ds)
                    array=ds.ReadAsArray()
                    dask_array=da.from_array(array,chunks=len(array)/2)
                    array_list.append(dask_array)
            

                
                stack = da.stack(array_list, axis=0)
                max_array=stack.max(axis=0).compute()
                
                #print(target_imgs[0])
                year_name=target_imgs[0].split('/')[3]
                try:
                    output_folder='../data/Modis_Fire_Date/'+year_name+'/max_tif/'
                    os.mkdir(output_folder)
                except:
                    pass

                s_srs = ds.GetProjectionRef()    
                osng = osr.SpatialReference ()

                #use last img to define name of the last product
                # sample : 'h35v10'
                folder_name=img.split('/')[-1].split('.')[-4]
                
                osng.SetFromUserInput ( s_srs )    
                geo_t = ds.GetGeoTransform ()    
                x_size = ds.RasterXSize # Raster xsize
                y_size = ds.RasterYSize # Raster ysize
                mem_drv= gdal.GetDriverByName( 'MEM')
                dest = mem_drv.Create('', x_size,y_size, 1)
                dest.SetGeoTransform( geo_t )
                dest.SetProjection( osng.ExportToWkt())
                dest.GetRasterBand(1).WriteArray(max_array)
                output_max_path=output_folder+'modis_MCD64A1_max_'+folder_name+'.tif'
                gdal.Warp(output_max_path, dest, format = 'GTiff')
                max_array=None
                dest=None
                ds=None
                mem_drv=None
                image_status=True
                #print(output_max_path)

            except:            
                image_status=False
                counter+=1
                if counter==10:
                    print(target_imgs[0] + ' couldnt be calculated')
                    print("Unexpected error:", sys.exc_info()[0])
                    break
                continue
    
                

CPU times: user 2min 55s, sys: 33.6 s, total: 3min 28s
Wall time: 4min 40s


##### Removing folders

In [45]:
import shutil
rm=glob.glob('../data/Modis_Fire_Date/*/*/tif')

In [47]:
for i in rm:
    #print(i)
    shutil.rmtree(i,ignore_errors=True)